# RFM Analysis

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/Aswath98/MarketingAnalytics/master/DataToModel.csv',index_col =[0])

## Create the RFM Table

In [22]:
import datetime as dt
NOW = dt.datetime(2013,12,31)

In [5]:
# Make the date_placed column datetime
train['Outlet_Establishment_Year'] = pd.to_datetime(train['Outlet_Establishment_Year'])

Create the RFM Table

In [30]:
rfmTable = train.groupby('Item_Identifier').agg({'Outlet_Establishment_Year': lambda x: (NOW - x.max()).days/365.25, # Recency
                                        'Outlet_Identifier': lambda x: len(x),      # Frequency
                                        'Item_Outlet_Sales': lambda x: x.sum()}) # Monetary Value

rfmTable['Outlet_Establishment_Year'] = rfmTable['Outlet_Establishment_Year'].astype(int)
rfmTable.rename(columns={'Outlet_Establishment_Year': 'recency', 
                         'Outlet_Identifier': 'frequency', 
                         'Item_Outlet_Sales': 'monetary_value'}, inplace=True)

In [31]:
rfmTable

,recency,frequency,monetary_value
Item_Identifier,,,
DRA12,43,6,11061.6012
DRA24,43,7,15723.5328
DRA59,43,8,20915.4412
DRB01,43,3,4554.0720
DRB13,43,5,12144.1920
DRB24,43,4,12202.7824
DRB25,43,6,9969.0234
DRB48,43,7,4360.3242
DRC01,43,6,5173.2660


## Determining RFM Quartiles

In [34]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [35]:
quantiles

,recency,frequency,monetary_value
0.25,43.0,4.0,6318.7749
0.50,43.0,5.0,10785.9600
0.75,43.0,6.5,16045.7800


Send quantiles to a dictionary, easier to use.

In [36]:
quantiles = quantiles.to_dict()

In [37]:
quantiles

{'frequency': {0.25: 4.0, 0.5: 5.0, 0.75: 6.5},
 'monetary_value': {0.25: 6318.7749, 0.5: 10785.96, 0.75: 16045.779999999999},
 'recency': {0.25: 43.0, 0.5: 43.0, 0.75: 43.0}}

## Creating the RFM segmentation table

In [38]:
rfmSegmentation = rfmTable

In [39]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [40]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [41]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [50]:
rfmSegmentation = rfmSegmentation.reset_index()

In [51]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111']

,Item_Identifier,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
2,DRA59,43,8,20915.4412,1,1,1,111
19,DRD15,43,7,17659.6792,1,1,1,111
34,DRE48,43,7,24437.5232,1,1,1,111
35,DRE49,43,9,24136.5816,1,1,1,111
41,DRF23,43,8,21107.1916,1,1,1,111
43,DRF27,43,8,17457.2760,1,1,1,111
77,DRI01,43,7,21727.7172,1,1,1,111
78,DRI03,43,8,22492.0556,1,1,1,111
90,DRJ11,43,7,20421.4176,1,1,1,111
113,DRL01,43,7,29913.0624,1,1,1,111


Which are the top 5 best items that bought high sales? by RFM Class (111) , we could find it 

In [53]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

,Item_Identifier,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
1210,FDY55,43,8,42661.8008,1,1,1,111
156,FDA15,43,8,41584.5364,1,1,1,111
1230,FDZ20,43,8,40185.0248,1,1,1,111
359,FDF05,43,8,36555.7490,1,1,1,111
148,FDA04,43,8,35741.4756,1,1,1,111
